<a href="https://colab.research.google.com/github/gazon1/Recco-challenge/blob/master/nn_Embedings_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import json
import pandas as pd
import numpy as np


import tqdm
import scipy.sparse as sp
from sklearn.metrics import average_precision_score as score
import time
from matplotlib import pyplot as plt

from sklearn import preprocessing
from itertools import chain

from keras.models import model_from_json
import keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, BatchNormalization, Dense,LSTM, CuDNNGRU
from keras.models import Sequential
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras.models import Model
from keras import optimizers

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer, StandardScaler
  
  

import itertools
from pprint import pprint

DATA_PATH = ''
%matplotlib inline

# For Google Colab only:
import sys
sys.path.append('/content/gdrive/My Drive/okko competition')
#from reco_utils.recommender.sar.sar_singlenode import SARSingleNode
from google.colab import drive
drive.mount('/content/gdrive')

DATA_PATH = 'gdrive/My Drive/Colab Notebooks/'
CODE_PATH = 'gdrive/My Drive/okko competition/code'

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18298470202223723924
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5217527777567535036
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 735202548726662114
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11105817994785811174
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.co

In [0]:
from keras.datasets import imdb
max_features = 10000
maxlen=20

(x_train, y_train), (x_test, y_test) = imdb.load_data(
num_words=max_features)
print(x_train.shape)
print(len(x_train[0]))
print(x_train[0][-20:])

x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)

(25000,)
218
[65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [0]:
x_train[0]

array([  65,   16,   38, 1334,   88,   12,   16,  283,    5,   16, 4472,
        113,  103,   32,   15,   16, 5345,   19,  178,   32], dtype=int32)

Фичи для фильма

In [0]:
with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
    catalogue = json.load(f)
    
catalogue = {int(k): v for k, v in catalogue.items()}
catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
# av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})

In [0]:
catalogue = catalogue[['element_uid','duration','feature_1','feature_2','feature_3','feature_4','feature_5','av_hot']]
scaler = StandardScaler()
X_cat = catalogue.values[:,1:]
X_cat = scaler.fit_transform(X_cat)
X_cat = X_cat[:, np.newaxis].reshape(-1,7,1)

In [0]:
X.shape

(10200, 7, 1)

LSTM AE для построения  embedding'a

In [0]:
#sequence = np.copy(movie_all_features)
n_in = 7
#sequence = .reshape((-1, n_in, 1))

# define model
model = Sequential()
model.add(LSTM(20, activation='relu', input_shape=(n_in,1)))
model.add(RepeatVector(n_in))
model.add(LSTM(20, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')

In [0]:
# ~ 20 epoch хватит для mse = 20
model.fit(X_cat, X_cat, epochs=5,verbose=1, shuffle=True)

Epoch 1/10
10200/10200 [==============================] - 12s 1ms/step - loss: 0.8530
Epoch 2/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.7618
Epoch 3/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.6754
Epoch 4/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.6014
Epoch 5/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.5022
Epoch 6/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.4031
Epoch 7/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.3396
Epoch 8/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.2817
Epoch 9/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.2089
Epoch 10/10
10200/10200 [==============================] - 11s 1ms/step - loss: 0.1707


Выделяем из LSTM AE только энкодер, который будет выдавать в ответ на фичи фильма вектор-ембеддинг

In [0]:
encoder = Model(inputs=model.inputs, outputs=model.layers[0].output)
plot_model(encoder, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat = encoder.predict(X_cat[np.newaxis, 0])
print(yhat.shape)
print(yhat)

(1, 20)
[[1.241389   0.0979873  0.5464926  0.00902695 0.23990184 0.16649489
  0.64022106 0.34092176 1.7395786  1.2301583  0.03461496 2.1898713
  0.69029355 5.567245   1.9777299  0.12397002 0.54865247 0.4374257
  0.00821178 0.68293196]]


Друие фичи для фильмов

In [0]:
import gc
from tqdm import tqdm
tqdm.pandas()
def get_sequences():
    import gc
    import tqdm

    #Hyperparametrs:
    WATCHED_TIME = 600 # выкидываем передачи, просмотренные меньше WATCHED_TIME. ЧТобы ничего не выкидывать - ставим 0
    MIN_RATING = 3 # ниже этого рейтинга все выкидываем`

    with open(os.path.join(DATA_PATH, 'catalogue.json'), 'r') as f:
        catalogue = json.load(f)

    catalogue = {int(k): v for k, v in catalogue.items()}
    catalogue = pd.DataFrame.from_dict(catalogue,'index').reset_index()
    # av_hot - нужно для валидации. Эта колонка равна 1, если фильм или сериал может быть потребленным
    catalogue['av_hot'] = catalogue['availability'].apply(lambda x: 1 if x == [] or 'subscription' in x else 0)
    catalogue = catalogue.rename(index=str, columns={'index':'element_uid'})



    transactions = pd.read_csv(
        os.path.join(DATA_PATH, 'transactions.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'consumption_mode': 'category',
            'ts': np.float64,
            'watched_time': np.uint64,
            'device_type': np.uint8,
            'device_manufacturer': np.uint8
        }
    )

    ratings = pd.read_csv(
        os.path.join(DATA_PATH, 'ratings.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64,
            'rating': np.uint8
        }
    )

    bookmarks = pd.read_csv(
        os.path.join(DATA_PATH, 'bookmarks.csv'),
        dtype={
            'element_uid': np.uint16,
            'user_uid': np.uint32,
            'ts': np.float64
        }
    )

    bookmarks['rating'] = 1
    _bookmarks = bookmarks[['rating', 'user_uid', 'element_uid','ts']]

    _ratings = ratings[ratings['rating']>5]
    _ratings['rating'] = 1

    _transaction = transactions[['element_uid', 'user_uid','ts','watched_time']]
    _transaction = _transaction[(_transaction['watched_time']>700)]
    _transaction['rating'] = 1

    all_ratings = pd.concat([ _transaction[['user_uid', 'element_uid', 'rating', 'ts']],_bookmarks[['user_uid', 'element_uid', 'rating', 'ts']]])
    all_ratings = pd.concat([all_ratings, _ratings[['user_uid', 'element_uid', 'rating','ts']]])
    all_ratings = all_ratings.drop_duplicates(['user_uid','element_uid'],keep='first')
    all_ratings = all_ratings.sort_values(by=['ts'])
    all_ratings= all_ratings.rename(index=str,columns={'element_uid':'itemID','user_uid':'userID','ts':'timestamp'})



    all_ratings.itemID = all_ratings.itemID.astype('category')
    all_ratings['categ_id'] = all_ratings.itemID.cat.codes + 1
    cat_to_element_uid = dict(zip(
        range(1, len(all_ratings.itemID.cat.categories) + 1),
        all_ratings.itemID.cat.categories
    ))


    try:
      del bookmarks, ratings, catalogue, transactions, _transactions, _ratings
    except NameError:
      pass
    gc.collect()


    tqdm.tqdm.pandas()
    sequences = all_ratings.groupby('userID')['categ_id'].progress_apply(list)

    max_features = all_ratings.categ_id.unique().size + 1

    del all_ratings
    gc.collect()

    sequences2use = sequences[sequences.apply(len) >= 5]

    maxlen = 18 # Length of sequences in X
    X = []
    y = []

    def slice_sequence(seq, num_slices):
        for i in range(1, num_slices):
            X.append(seq[-(i+maxlen): -i])
            y.append(seq[-i])

    for seq in tqdm.tqdm(sequences2use):
        if len(seq) <= 5:
            slice_sequence(seq, 2)
        elif len(seq) <= 6:
            slice_sequence(seq, 3)
        elif len(seq) <= 8:
            slice_sequence(seq, 4)
        elif len(seq) <= 12:
            slice_sequence(seq, 6)
        elif len(seq) <= 16:
            slice_sequence(seq, 8)
        elif len(seq) <= 20:
            slice_sequence(seq, 11)
        elif len(seq) <= 26:
            slice_sequence(seq, 16)
        else:
            slice_sequence(seq, 23)

    X = pad_sequences(X, maxlen=maxlen)
    y = np.array(y)

    
    le = preprocessing.LabelEncoder()

    _X = np.ravel(X)
    all_movies = list(set(y).union(set(_X)))
    le.fit(all_movies)

    #теперь в _y будут числа от 0 до кол-во фильмов (max_features)
    _y = le.transform(y)

    _X = le.transform(_X)
    _X = _X.reshape((-1,18))
    
    
    return _X, _y, cat_to_element_uid

In [0]:
%%time
sequences = get_sequences()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

100%|██████████| 347419/347419 [00:07<00:00, 44722.98it/s]

CPU times: user 1min 38s, sys: 3.71 s, total: 1min 41s
Wall time: 1min 42s


Feature engeniring

In [0]:
X, y,cat_to_el = sequences[0],sequences[1],sequences[2]
del sequences
gc.collect()

98

In [0]:
from keras.layers import Input, Embedding, SpatialDropout1D, Dropout, Dense,LSTM, CuDNNGRU#, CuDNNLSTM, CuDNNGRU
from keras.models import Model
from keras import backend as K

import keras
K.tensorflow_backend._get_available_gpus()

# Let's set random seed
import tensorflow as tf
tf.set_random_seed(42)
np.random.seed(42)

In [0]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [0]:
max_features = catalogue.shape[0] + 1
embed_size = 128
maxlen = 18
def lstm129():
    inp = Input(shape=(maxlen,20))
   # x = Embedding(max_features, embed_size)(inp)

    x = CuDNNGRU(128,return_sequences=False)(inp)
    x = Dropout(0.02)(x)
    outp = Dense(max_features, activation="softmax")(x)
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop',
                  metrics=['sparse_categorical_accuracy'])
    return model

In [0]:
x = X.reshape(-1,18,1)
x.shape

(3399095, 18, 1)

In [0]:
encoder = Model(inputs=model.inputs, outputs=model.layers[0].output)
plot_model(encoder, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
del X_gen
gc.collect()

3209

In [0]:
#yhat = encoder.predict(x[0][0][np.newaxis, 0])
#yhat
#[encoder.predict(el[np.newaxis, 0]) for el in x[0]]
def get_em(x):
 # print(x.shape)
  return np.array([encoder.predict(X_cat[np.newaxis, el[0]]).tolist() for el in x]).reshape(-1,20)

X_gen = np.zeros((int(X.shape[0]/2),maxlen,20))

for i in tqdm(range(X.shape[0])):
  X_gen[i] = get_em(np.array(X[i]).reshape(maxlen,1))
  




  0%|          | 373/3399095 [00:49<125:22:22,  7.53it/s]

KeyboardInterrupt: ignored

In [0]:
#encoder.predict()
def tem_func(x):
 # print(cat_to_el[x])
  return X_cat[cat_to_el[x]]
  


temp_seq
#X_cat.shape
#x[0]
del temp_seq
gc.collect()


41

In [0]:
encoder.predict(X_cat[np.newaxis, el[0]]).tolist() for el in x

SyntaxError: ignored

In [0]:
x[0]

array([[6934],
       [4558],
       [7572],
       [6029],
       [2607],
       [3448],
       [5447],
       [1806],
       [5096],
       [6635],
       [4163],
       [5711],
       [3865],
       [8119],
       [3315],
       [  43],
       [6314],
       [3069]])

In [0]:
model_gen = lstm129()
model_gen.fit(X_gen[:500], y[:500], batch_size=2048, epochs=10, verbose=True, validation_split=0.01, shuffle=True)

Train on 495 samples, validate on 5 samples
Epoch 1/10
495/495 [==============================] - 3s 6ms/step - loss: 9.2336 - sparse_categorical_accuracy: 0.0000e+00 - val_loss: 9.0932 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/10
495/495 [==============================] - 0s 88us/step - loss: 9.0031 - sparse_categorical_accuracy: 0.0061 - val_loss: 8.9147 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/10
495/495 [==============================] - 0s 75us/step - loss: 8.6950 - sparse_categorical_accuracy: 0.0081 - val_loss: 8.6659 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/10
495/495 [==============================] - 0s 79us/step - loss: 8.2622 - sparse_categorical_accuracy: 0.0061 - val_loss: 8.3672 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/10
495/495 [==============================] - 0s 77us/step - loss: 7.7199 - sparse_categorical_accuracy: 0.0081 - val_loss: 8.1091 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/10
495/495 [========